In [28]:
import pandas as pd
from sqlalchemy import create_engine
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

In [2]:
conn_details = {
    "host": "postgresql-hackathon2024.alwaysdata.net",
    "dbname": "hackathon2024_2",
    "user": "hackathon2024_reader",
    "password": "hackathon_2_reader"
}

In [3]:
db_url = f"postgresql+psycopg2://{conn_details['user']}:{conn_details['password']}@{conn_details['host']}/{conn_details['dbname']}"

In [4]:
engine = create_engine(db_url)

In [5]:
# 1 hackathon
df = pd.read_sql_query("SELECT * FROM olympic_hosts", engine)
df=pd.concat([df, pd.DataFrame({"game_slug": ["paris-2024"],"game_location": ["France"],"game_name": ["Paris 2024"],"game_season": ["Summer"],"game_year": [2024]})], ignore_index=True)
# # les JO 2024 de France n'existe pas dans cette base de donées
filtered_df=df[df['game_location'] == 'France']
fig=px.pie(names=['Ete', 'Hiver'], values=[len(filtered_df[filtered_df['game_season'] == 'Summer']), len(filtered_df[filtered_df['game_season'] == 'Winter'])], title='Nombre de JO organiser en Ete et Hiver par la France')
fig.show(renderer='iframe_connected')

In [6]:
# 2 hackathon
fig = px.bar(df['game_location'].value_counts().reset_index(), x='game_location', y='count', title='Classement des pays qui a organisé le plus de JO')
fig.show(renderer='iframe_connected')

In [7]:
# 3 hackathon
df3=df[df['game_year'] != 'NaN']
df3['game_year'] = pd.to_numeric(df3['game_year'], errors='coerce')
res=df3.sort_values(by='game_year').iloc[0]
print( "les Jo sont nés en "+res['game_season']+" "+str(res['game_year'])+" à "+res['game_location'])

les Jo sont nés en Summer 1896 à Greece


In [8]:
# 4 hackathon
filtered_game=df[df['game_location']=='France']
filtered_game=filtered_game[filtered_game['game_year'] != 'NaN']
filtered_game['game_year'] = pd.to_numeric(filtered_game['game_year'], errors='coerce')
slug=filtered_game[filtered_game['game_year']==1900].iloc[0]['game_slug']
df4=pd.read_sql_query("SELECT * FROM olympic_medals", engine)
filtered_df4=df4[df4['slug_game']== slug]
print("les femmes "+ ("peuvent " if len(filtered_df4[filtered_df4['event_gender']== "Women"]) !=0 else "ne peuvent pas ")+"participer au JO")

les femmes peuvent participer au JO


In [9]:
# 5 hackathon
df5=pd.read_sql_query("SELECT om.athlete_full_name,olympic_hosts.game_year,olympic_hosts.game_season FROM olympic_medals as om inner join olympic_hosts on om.slug_game=olympic_hosts.game_slug ", engine)
df5=df5[df5['athlete_full_name'] != 'NaN']
df5=df5[df5.duplicated(subset='athlete_full_name', keep=False)].groupby(['athlete_full_name','game_season','game_year']).sum().reset_index().groupby(['athlete_full_name','game_season']).agg({
    'game_year': ', '.join
}).reset_index()
df5=df5[df5.duplicated(subset=['athlete_full_name'], keep=False)]
df5['message'] = "en "+df5['game_season'] +" de "+df5['game_year']
df5 = df5.drop(columns=['game_season','game_year'])
df5.groupby(['athlete_full_name']).agg({
    'message': ' et '.join}).reset_index()
for _index, row in df5.groupby(['athlete_full_name']).agg({'message': ' et '.join}).reset_index().iterrows():
    print(row['athlete_full_name']+" a gagner des medailles "+ row['message']+"\n")

Christa LUDING-ROTHENBURGER a gagner des medailles en Summer de 1988 et en Winter de 1984, 1988, 1992

Clara HUGHES a gagner des medailles en Summer de 1996 et en Winter de 2002, 2006, 2010

Gillis GRAFSTRÖM a gagner des medailles en Summer de 1920 et en Winter de 1924, 1928, 1932

Gunnar LARSSON a gagner des medailles en Summer de 1972 et en Winter de 1968



In [10]:
# 6 hackathon
df6=pd.read_sql_query("SELECT game_season,game_year FROM olympic_hosts",engine)
df6['game_year'] = pd.to_numeric(df6['game_year'], errors='coerce')
df6
# df6.groupby('game_year').agg({
#     'game_season':', '.join
# }).reset_index()
year_counts=df6['game_year'].value_counts().sort_index()
df6_1=df6[(df6['game_year'] >= 1924) & (df6['game_year'] <= 1992)]
grouped = df6_1.groupby(['game_season', 'game_year']).size().unstack(fill_value=0)
grouped = grouped.T
# Create the bar chart
fig = go.Figure(data=[
    go.Bar(name='Winter', x=grouped.index, y=grouped['Winter']),
    go.Bar(name='Summer', x=grouped.index, y=grouped['Summer'])
])

fig.update_layout(
    title='Number of occurrences by year and season',
    xaxis_title='Années',
    barmode='group',
    template='plotly_white',
    yaxis=dict(
        showticklabels=False
    ),
    xaxis=dict(
        tickmode='linear',
        tick0=1924,
        dtick=4
    )
)
fig.show(renderer='iframe_connected')

In [11]:
df6_2=df6[(df6['game_year'] > 1992)]
grouped = df6_2.groupby(['game_season', 'game_year']).size().unstack(fill_value=0)
grouped = grouped.T
# Create the bar chart
fig2 = go.Figure(data=[
    go.Bar(name='Winter', x=grouped.index, y=grouped['Winter']),
    go.Bar(name='Summer', x=grouped.index, y=grouped['Summer'])
])

fig2.update_layout(
    title='Number of occurrences by year and season',
    xaxis_title='Années',
    barmode='group',
    template='plotly_white',
    yaxis=dict(
        showticklabels=False
    ),
    xaxis=dict(
        tickmode='linear',
        tick0=1994,
        dtick=2
    )
)
fig2.show(renderer='iframe_connected')

In [12]:
# 7 hackathon
df7=pd.read_sql_query("SELECT om.event_title,om.medal_type,om.athlete_full_name,oh.game_year FROM olympic_medals as om inner join olympic_hosts as oh on om.slug_game=oh.game_slug",engine)
df7=df7[df7['athlete_full_name'] == 'Johnny WEISSMULLER']
df7['message'] = "1 medaille "+df7['medal_type']+" en "+df7['event_title'] +" en "+df7['game_year']
df7=df7.groupby('athlete_full_name').agg({'message':', '.join}).reset_index().iloc[0]
print(df7['athlete_full_name']+" a obtenu "+df7['message'])

Johnny WEISSMULLER a obtenu 1 medaille GOLD en 100m freestyle men en 1928, 1 medaille GOLD en 400m freestyle men en 1924, 1 medaille GOLD en 100m freestyle men en 1924


In [13]:
# 8 hackathon
df8=pd.read_sql_query("SELECT om.event_gender,om.country_name,om.slug_game FROM olympic_medals as om",engine)
df8=df8.groupby(['country_name','slug_game']).agg({'event_gender':lambda values: "Women" if "Women" in ','.join(values).split(',') else "Men"}).reset_index()
df8=df8.groupby('slug_game').agg({'event_gender':lambda values:"No" if "Men" in ','.join(values).split(',') else "Yes"}).reset_index()
df8=df8[df8['event_gender']=="Yes"].iloc[0]
print("le championnat de "+df8['slug_game']+" est le 1ers JO durant lesquels tous les pays participants ont envoyé des athlètes de sexe féminin")

le championnat de grenoble-1968 est le 1ers JO durant lesquels tous les pays participants ont envoyé des athlètes de sexe féminin


In [14]:
# 8 hackathon
df = pd.read_sql_query("SELECT * FROM olympic_medals", engine)
df=df[df['slug_game'] == 'london-2012']
pays_filtered=df[df['event_gender'].isin(['Women','Mixed'])]
print("le championnat de londre 2012 avait "+str(len(pays_filtered.drop_duplicates(subset=['country_name'])))+" femme de different pays pour "+str(len(df.drop_duplicates(subset=['country_name'])))+" pays")

le championnat de londre 2012 avait 57 femme de different pays pour 81 pays


In [15]:
# 9 hackathon
df9=pd.read_sql_query("SELECT slug_game	,event_title FROM olympic_medals",engine)
df9 = df9.drop_duplicates(subset=['slug_game','event_title'])
dp=df9.duplicated('event_title')
df9=df9[~dp]
df9[df9['slug_game']=='paris-1900']
print("les  sport qui sont aparue 1 seule fois au JO de paris 1900 sont: \n"+'\n'.join(["- "+row['event_title'] for _index, row in df9[df9['slug_game']=='paris-1900'].iterrows()]))

les  sport qui sont aparue 1 seule fois au JO de paris 1900 sont: 
- foil masters men
- pe masters men
- sabre masters men
- pe amateurs and masters men
- 25 kilometres men
- Course De Primes Men
- army rifle 300m kneeling men
- 4000m steeplechase men
- Championnat Du Monde Men
- au cordon dor 33m men
- au chapelet 50m men
- au cordon dor 50m men
- au chapelet 33m men
- sur la perche à la herse men
- sur la perche à la pyramide men
- high jump mixed
- long jump individual mixed
- individual 2 balls men
- individual 1 ball men
- 1000m freestyle men
- 4000m freestyle men
- 200m obstacle event men
- underwater swimming men


In [16]:
# 10 hackathon
df10=pd.read_sql_query("SELECT * FROM olympic_hosts",engine)
df10['game_year'] = pd.to_numeric(df10['game_year'], errors='coerce')
df10=df10.groupby('game_location').agg({'game_year':lambda values:', '.join([str(e) for e in values])}).reset_index()
df10.sort_values(by='game_year')

,game_location,game_year
12,Greece,"1896, 2004"
21,Sweden,1912
3,Belgium,1920
16,Netherlands,1928
10,Germany,"1936, 1936"
11,Great Britain,"1948, 1908, 2012"
22,Switzerland,"1948, 1928"
8,Finland,1952
17,Norway,"1952, 1994"
1,"Australia, Sweden",1956


In [17]:
# 11 hackathon
df11=pd.read_sql_query("SELECT ore.athlete_url,ore.athlete_full_name,ore.event_title,ore.discipline_title,ore.country_name,oh.game_season,oh.game_year FROM olympic_results as ore inner join olympic_hosts as oh on ore.slug_game=oh.game_slug",engine)
df11=df11[df11['game_season']=='Summer']
df11[df11['game_year']=='2016']
print("il y a "+str(len(df11.drop_duplicates(subset=['athlete_full_name','athlete_url'])))+" athletes de "+str(len(df11.drop_duplicates(subset=['country_name'])))+" pays different pour "+str(len(df11.drop_duplicates(subset=['discipline_title'])))+" sport different dans un total de "+str(len(df11.drop_duplicates(subset=['event_title'])))+" epreuves diferentes")

il y a 11274 athletes de 213 pays different pour 42 sport different dans un total de 446 epreuves diferentes


In [18]:
df = pd.read_sql_query("SELECT medal_type,country_name FROM olympic_medals", engine)
df['medal_type']=pd.to_numeric(df['medal_type'].replace({'GOLD': '3', 'SILVER': '2','BRONZE': '1'}), errors='coerce')
df=df.groupby(['country_name']).agg({'medal_type': lambda x: sum(x)}).reset_index()
fig = px.bar(df.sort_values(by='medal_type' ,ascending=False).head(30), x='country_name', y='medal_type', title='Top 30 des pays avec le plus de médaille des JO')
fig.update_layout(
    xaxis_title='Pays',
    barmode='group',
    template='plotly_white',
    yaxis=dict(
        showticklabels=False
    )
)
fig.show(renderer='iframe_connected')

In [24]:
df = pd.read_sql_query("SELECT om.medal_type,om.country_name,oh.game_season FROM olympic_medals as om inner join olympic_hosts as oh on om.slug_game=oh.game_slug", engine)
df=df[df['game_season']=='Summer']
df['medal_type']=pd.to_numeric(df['medal_type'].replace({'GOLD': '3', 'SILVER': '2','BRONZE': '1'}), errors='coerce')
df=df.groupby(['country_name']).agg({'medal_type': lambda x: sum(x)}).reset_index()
fig = px.bar(df.sort_values(by='medal_type' ,ascending=False).head(30), x='country_name', y='medal_type', title='Top 30 des pays avec le plus de médaille des JO d\'ete')
fig.update_layout(
    xaxis_title='Pays',
    barmode='group',
    template='plotly_white',
    yaxis=dict(
        showticklabels=False
    )
)
fig.show(renderer='iframe_connected')

In [26]:
df = pd.read_sql_query("SELECT om.medal_type,om.country_name,oh.game_season FROM olympic_medals as om inner join olympic_hosts as oh on om.slug_game=oh.game_slug", engine)
df=df[df['game_season']=='Winter']
df['medal_type']=pd.to_numeric(df['medal_type'].replace({'GOLD': '3', 'SILVER': '2','BRONZE': '1'}), errors='coerce')
df=df.groupby(['country_name']).agg({'medal_type': lambda x: sum(x)}).reset_index()
fig = px.bar(df.sort_values(by='medal_type' ,ascending=False).head(30), x='country_name', y='medal_type', title='Top 30 des pays avec le plus de médaille des JO d\'hiver')
fig.update_layout(
    xaxis_title='Pays',
    barmode='group',
    template='plotly_white',
    yaxis=dict(
        showticklabels=False
    )
)
fig.show(renderer='iframe_connected')

In [25]:
df = pd.read_sql_query("SELECT country_name,athlete_full_name FROM olympic_medals", engine)
df.groupby(['athlete_full_name']).sum().reset_index()
df['nombre'] = 1
fig = px.bar(df.groupby(['country_name']).agg({'nombre': lambda x: sum(x)}).reset_index().sort_values(by='nombre' ,ascending=False).head(30), x='country_name', y='nombre', title='Top 30 des pays avec le plus d\'athletes des JO')
fig.update_layout(
    xaxis_title='Pays',
    barmode='group',
    template='plotly_white',
    yaxis=dict(
        showticklabels=False
    )
)
fig.show(renderer='iframe_connected')

In [29]:
df=pd.read_sql_query("SELECT ore.rank_position,ore.country_name,ore.country_3_letter_code,oh.game_year,oh.game_season FROM olympic_results as ore inner join olympic_hosts as oh on oh.game_slug=ore.slug_game", engine)
annee='2022'
df=df[df['game_year']==annee]
df=df[df['game_season']=='Winter']
df['rank_position']=pd.to_numeric(df['rank_position'], errors='coerce')
df=df[~df['rank_position'].isna()]
df=df.groupby(['country_name','country_3_letter_code']).agg({'rank_position':lambda x:sum(x)}).reset_index()
df=df.sort_values(by='rank_position' ,ascending=False).head(30)
df['pib'] = np.nan
dfpid = pd.read_csv('./pib 1.csv')
dfpid['Country Name,"Country Code","Indicator Name","Indicator Code","1960","1961","1962","1963","1964","1965","1966","1967","1968","1969","1970","1971","1972","1973","1974","1975","1976","1977","1978","1979","1980","1981","1982","1983","1984","1985","1986","1987","1988","1989","1990","1991","1992","1993","1994","1995","1996","1997","1998","1999","2000","2001","2002","2003","2004","2005","2006","2007","2008","2009","2010","2011","2012","2013","2014","2015","2016","2017","2018","2019","2020","2021","2022","2023",']=dfpid['Country Name,"Country Code","Indicator Name","Indicator Code","1960","1961","1962","1963","1964","1965","1966","1967","1968","1969","1970","1971","1972","1973","1974","1975","1976","1977","1978","1979","1980","1981","1982","1983","1984","1985","1986","1987","1988","1989","1990","1991","1992","1993","1994","1995","1996","1997","1998","1999","2000","2001","2002","2003","2004","2005","2006","2007","2008","2009","2010","2011","2012","2013","2014","2015","2016","2017","2018","2019","2020","2021","2022","2023",'].str.replace('"', '')
df_split=dfpid['Country Name,"Country Code","Indicator Name","Indicator Code","1960","1961","1962","1963","1964","1965","1966","1967","1968","1969","1970","1971","1972","1973","1974","1975","1976","1977","1978","1979","1980","1981","1982","1983","1984","1985","1986","1987","1988","1989","1990","1991","1992","1993","1994","1995","1996","1997","1998","1999","2000","2001","2002","2003","2004","2005","2006","2007","2008","2009","2010","2011","2012","2013","2014","2015","2016","2017","2018","2019","2020","2021","2022","2023",'].str.split(',', expand=True)
compt=0
for i in df_split.columns:
    if i>3:
        df_split.rename(columns={i: str(1960+compt)}, inplace=True)
        compt+=1
new_df = df_split[[1, annee]]
df['pib'] = df.apply(lambda x: new_df.loc[new_df[1] == x['country_3_letter_code'], annee].iloc[0] if x['country_3_letter_code'] in new_df[1].values else None, axis=1)
df=df.dropna(subset=['pib'])
df['pib']=pd.to_numeric(df['pib'], errors='coerce')
df = df.sort_values(by='rank_position', ascending=False)

fig = px.bar(df, x='country_name', y='pib', title='Classement des pays au JO 2022 avec leur PIB', text='pib')
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(
    xaxis_title='Pays',
    yaxis_title='PIB',
    template='plotly_white'
)
fig.show(renderer='iframe_connected')